This notebook adapts the [Captum tutorial for question answering](https://captum.ai/tutorials/Bert_SQUAD_Interpret) and refactors it into the longformer sequence classification task. Specifically, this notebook focuses on using the model's embeddings to get token attributions for the examples of your choice, or the entire dataset if needed. By doing so, we can visualize which tokens have the most influence in the model's prediction, and find out the k tokens with the most influence at helping the model predict correctly as well as incorrectly.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import dependencies

In [ ]:
pip install transformers --quiet

In [ ]:
pip install captum --quiet

In [ ]:
pip install datasets --quiet

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch
import pandas as pd

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Import model

Here we are importing the model and tokenizer and letting the model use our GPU to run. Please change model path, and tokenizer to whichever one you wish to use.

In [ ]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model

# load model
test = torch.load("/content/drive/MyDrive/cogs402longformer/fakeclinicalnotes/models/full_augmented_lr2e-5_dropout3_10_trained_threshold.pt")
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', state_dict=test['state_dict'], num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['dense.weight', 'longformer_model.encoder.layer.10.attention.output.LayerNorm.weight', 'longformer_model.encoder.layer.7.attention.output.dense.bias', 'longformer_model.encoder.layer.4.attention.self.query.bias', 'longformer_model.encoder.layer.0.attention.self.key.weight', 'longformer_model.encoder.layer.9.attention.self.query.weight', 'longformer_model.encoder.layer.5.attention.self.key_global.bias', 'longformer_model.encoder.layer.6.attention.self.query.weight', 'longformer_model.encoder.layer.3.attention.self.key_global.weight', 'longformer_model.encoder.layer.10.attention.self.query.weight', 'longformer_model.encoder.layer.4.output.dense.weight', 'longformer_model.encoder.layer.10.output.dense.weight', 'longformer_model.encoder.layer.3.intermediate.dense.bias', 'longformer_model.encoder.layer.9.intermediate.dense.bias', 'longformer_model.encode

Create functions that give us the input ids and the position ids for the text we want to examine along with the baselines for integrated gradients.

In [ ]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [ ]:
max_length = 2046
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):
    text = text.lower()
    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)

    #taken from the longformer implementation
    mask = input_ids.ne(ref_token_id).int()
    incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask
    position_ids = incremental_indices.long().squeeze() + ref_token_id

    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    position_ids = position_ids[:, :seq_length]
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

### Import Dataset

Here we import the papers dataset

In [ ]:
from datasets import load_dataset
import numpy as np
# cogs402_ds = load_dataset("danielhou13/cogs402datafake")["train"]

ds = pd.read_csv("/content/drive/MyDrive/cogs402longformer/fakeclinicalnotes/data/fake_notes.csv")
dataset = datasets.Dataset.from_pandas(ds)
cogs402_ds = dataset.train_test_split(test_size=0.20)
cogs402_ds = cogs402_ds['train']

  0%|          | 0/1 [00:00<?, ?it/s]

Here we import the news dataset

In [ ]:
# cogs402_ds = load_dataset("danielhou13/cogs402dataset2")["validation"]

## Getting the Attributions

A custom forward function that returns the softmaxed logits, which are the class probabilities that the model uses for prediction.

In [ ]:
def predict(inputs, position_ids=None, attention_mask=None):
    output = model(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    return output.logits

In [ ]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None, attention_mask=None):
    preds = predict(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask
                   )
    return torch.softmax(preds, dim = 1)

A helper function to summarize attributions for each word token in the sequence.

In [ ]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.linalg.norm(attributions)
    return attributions

Perform Layer Integrated Gradients using the longformer's embeddings.

In [ ]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

This function will let us get the example and the baseline inputs in order to perform integrated gradients, and add the attributions to our visualization tool. Additionally, we will add the attributions and tokens for each example into an array so we can use them when we want to further examine the attributions scores for each example. More information about the integrated gradients function can be found [here](https://captum.ai/api/layer.html#layer-integrated-gradients).

In [ ]:
vis_data_records = []
all_attributions = {}
all_tokens = {}
all_deltas = {}

In [ ]:
# Takes in dataset and example number
def get_token_attributions(dataset, example):
  text = dataset['text'][example]
  label = dataset['labels'][example]
  # get the inputs, position ids, attention mask, and the baselines
  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
  attention_mask = construct_attention_mask(input_ids)

  #get the tokens
  indices = input_ids[0].detach().tolist()
  all_tokens_curr = tokenizer.convert_ids_to_tokens(indices)
  all_tokens[str(example)] = all_tokens_curr

  #perform integrated gradients
  attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True,
                                    additional_forward_args=(position_ids, attention_mask),
                                    target=1,
                                    n_steps=500,
                                    internal_batch_size = 2)

  # We want one value for every token.
  attributions_sum = summarize_attributions(attributions)

  # store the values in our dictionary
  all_attributions[str(example)] = attributions_sum
  all_deltas[str(example)] = attributions_sum

  # get the score for our visualization
  score = predict(input_ids, position_ids, attention_mask)

  all_tokens_curr = [x.replace('Ġ', '') for x in all_tokens_curr]
  # storing couple samples in an array for visualization purposes
  # requires array of attributions, prediction score, predicted class, true class 
  # the label you want your attributions to associate positive with, the attribution score
  # the tokens, and the delta if you have it.
  vis_data_records.append(viz.VisualizationDataRecord(
                        attributions_sum,
                        torch.softmax(score, dim = 1).max(),
                        torch.argmax(torch.softmax(score, dim = 1)),
                        label,
                        str(1),
                        attributions_sum.sum(),       
                        all_tokens_curr,
                        delta)
  )

Here we are taking some examples from the Papers datasets.

In [ ]:
get_token_attributions(cogs402_ds, 7)

This function allows us to display our attributions in a manner that is easy to read. We can see the attributions of the word overlayed on top of their respective token. The green colour represents positive attributions (i.e. the model is attributing this token to influential for predicting the positive class) while the red colour represents negative attributions. 

In [ ]:
# # storing couple samples in an array for visualization purposes
# score_vis = viz.VisualizationDataRecord(
#                         attributions_sum,
#                         torch.softmax(score, dim = 1).max(),
#                         torch.argmax(torch.softmax(score, dim = 1)),
#                         label,
#                         str(1),
#                         attributions_sum.sum(),       
#                         all_tokens,
#                         delta)

print('\033[1m', 'Visualization For Score', '\033[0m')
_ = viz.visualize_text(vis_data_records)

 Visualization For Score 


## Further Examination of the Attributions

Next we might want to look in-depth about the attribution scores for each token of an example. We saved the attributions for the examples we looked at above, so we can easily retrieve the attributions. We also grab the examples because we want to know what tokens the attributions are associated with.

Both lists are of shape: (seq_len)

In [ ]:
example = 7
attributions_sum = all_attributions[f"{example}"]
all_tokens2 = all_tokens[f"{example}"]

These functions return which words had the strongest (most positive and most negative) attributions. Change the number of tokens you wish to visualize for your needs. It takes in the attributions and the tokens we grabbed in the previous cell and returns 3 lists: the topk (or bottomk) attributions, their respective token and their position.

Note: Remember that the attributions are with respect to the positive class, so the most impact tokens that helped the model predict the negative class will be in the botk attributed tokens.

In [ ]:
def get_topk_attributed_tokens(attrs, all_tokens, k=20):
    values, indices = torch.topk(attrs, k)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [ ]:
def get_botk_attributed_tokens(attrs, all_tokens, k=20):
    values, indices = torch.topk(attrs, k, largest=False)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

Convert the values, index of the values, and the token into a pandas Dataframe for visualization. It will be sorted by highest value for attributions to lowest. Alternatively, if youre looking for the most negative attributions, it goes from lowest to highest.

In [ ]:
top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_sum.cpu(), all_tokens2)
bot_words_start, bot_words_val_start, bot_word_ind_start = get_botk_attributed_tokens(attributions_sum.cpu(), all_tokens2)

df_high = pd.DataFrame({'Word': top_words_start, 'index':top_word_ind_start, 'attribution': top_words_val_start})

df_low = pd.DataFrame({'Word': bot_words_start, 'index':bot_word_ind_start, 'attribution': bot_words_val_start})
# df_start.style.apply(['cell_ids: False'])

# ['{}({})'.format(token, str(i)) for i, token in enumerate(all_tokens)]

Here we display our top k positively and negatively attributed tokens for our example.

In [ ]:
df_high['Word'] = df_high['Word'].str.replace('Ġ', '')
df_high

,Word,index,attribution
0,family,430,0.063025
1,threat,1619,0.061966
2,ben,214,0.060865
3,name,331,0.057714
4,you,1502,0.057316
5,discharged,1205,0.056924
6,name,318,0.056194
7,happen,2018,0.054360
8,1,325,0.054011
9,from,1330,0.053674


In [ ]:
df_low['Word'] = df_low['Word'].str.replace('Ġ', '')
df_low

,Word,index,attribution
0,plan,1966,-0.078297
1,plan,1122,-0.069454
2,computer,448,-0.069208
3,day,530,-0.068755
4,cycl,248,-0.066530
5,easier,1953,-0.066436
6,diagnosis,268,-0.064512
7,misogyn,1527,-0.061235
8,diagnosis,286,-0.060768
9,would,646,-0.059901


In [ ]:
d = {"tokens":all_tokens2, "attribution":attributions_sum[:len(all_tokens2)].cpu()}

We notice that there are many repeating tokens in each example that have different positions. While we might want to know how the position plays into the attributions, if we want to know strictly based on the tokens itself, we can add all the duplicate tokens together to get the aggregate attribution for each token. Therefore, we aggregate the attributions strictly based on token type.

In [ ]:
df_attrib = pd.DataFrame(d)
aggregation_functions = {'attribution': 'sum'}
df_new = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_functions)

In [ ]:
highest_attrib_tokens = df_new.sort_values(by=['attribution'], ascending=False).reset_index()
highest_attrib_tokens['tokens'] = highest_attrib_tokens['tokens'].str.replace('Ġ', '')
highest_attrib_tokens[:10]

,tokens,attribution
0,.,0.678344
1,he,0.606769
2,was,0.605042
3,",",0.569732
4,mg,0.453486
5,in,0.378450
6,,0.346632
7,on,0.299068
8,one,0.281195
9,1,0.280139


In [ ]:
lowest_attrib_tokens = df_new.sort_values(by=['attribution']).reset_index()
lowest_attrib_tokens['tokens'] = lowest_attrib_tokens['tokens'].str.replace('Ġ', '')
lowest_attrib_tokens[:10]

,tokens,attribution
0,that,-1.424330
1,with,-1.056753
2,and,-0.619182
3,a,-0.586632
4,the,-0.580131
5,his,-0.566687
6,of,-0.559768
7,very,-0.545634
8,is,-0.538290
9,ayne,-0.477179


## Masking the stopwords and non-alpha tokens

There may be some stopwords or punctuations in our top attributed tokens, so now that we have the list of the highest and lowest, we can identify interesting keywords.

In [ ]:
import nltk
from transformers import AutoTokenizer
nltk.download('stopwords')
tokenizer2 = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', add_prefix_space=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.corpus import stopwords
all_stopwords = stopwords.words('english')
all_stopwords.append(" ")
stopwords = set(tokenizer2.tokenize(all_stopwords, is_split_into_words =True))
stopwords.update(all_stopwords)
print(stopwords)

{'their', 'Ġas', 'each', 'Ġbeing', 'Ġon', 'Ġbe', 'with', 'Ġis', 'Ġeach', 'Ġthat', "she's", 'are', 'my', 'at', 'through', 'should', 'Ġmy', 'Ġwasn', 'Ġwhy', 'Ġfew', "shouldn't", 'so', 'will', 'me', 'shouldn', 'an', 'if', 'aren', 'in', 'Ġand', 'after', 'can', 'haven', 'shan', 'hers', 'Ġre', "wouldn't", 'over', "that'll", 'off', 'Ġve', 'Ġaren', 'not', 'Ġtheir', 'there', 'Ġonce', "hadn't", 'too', 'Ġmyself', 'has', "'s", 'Ġdid', 'Ġthen', 'him', 'Ġ', 'Ġuntil', 'from', 'it', 'Ġagain', "it's", 'those', 'Ġshe', 'own', 'Ġout', 'Ġmore', 'Ġwill', 'Ġno', 'because', 'that', 'Ġbecause', 'once', 'out', 'during', 'Ġwho', 'Ġo', 'Ġat', "didn't", 'we', 'Ġtheirs', 'Ġisn', "doesn't", 'Ġy', 'them', 'then', 'a', "wasn't", 're', 'up', 'theirs', 'be', 'Ġwhere', 'Ġall', 'Ġthis', 'into', 'Ġwhom', 'Ġor', 'Ġher', 'Ġt', 'both', 'do', 'Ġdoesn', 'Ġhim', 'have', 'this', 'wasn', 'Ġthese', 'yourself', 'how', 'Ġin', 'Ġwhich', "haven't", "shan't", 'Ġother', 'your', 'he', 'Ġwon', 'to', 'Ġhas', 'here', 'doing', 'below', 'Ġthr

Lets see the per-token attributions again

In [ ]:
df_high[(df_high['Word'].str.isalpha()) & ~(df_high['Word'].isin(stopwords))][:10].reset_index(drop=True)

,Word,index,attribution
0,family,430,0.063025
1,threat,1619,0.061966
2,ben,214,0.060865
3,name,331,0.057714
4,discharged,1205,0.056924
5,name,318,0.056194
6,happen,2018,0.054360
7,aggression,1098,0.053639
8,mg,1993,0.053250
9,take,1948,0.051977


In [ ]:
df_low[(df_low['Word'].str.isalpha()) & ~(df_low['Word'].isin(stopwords))][:10].reset_index(drop=True)

,Word,index,attribution
0,plan,1966,-0.078297
1,plan,1122,-0.069454
2,computer,448,-0.069208
3,day,530,-0.068755
4,cycl,248,-0.066530
5,easier,1953,-0.066436
6,diagnosis,268,-0.064512
7,misogyn,1527,-0.061235
8,diagnosis,286,-0.060768
9,would,646,-0.059901


Here we have the aggregate attributions for the example

In [ ]:
highest_attrib_tokens[(highest_attrib_tokens['tokens'].str.isalpha()) & ~(highest_attrib_tokens['tokens'].isin(stopwords))][:10].reset_index(drop=True)

,tokens,attribution
0,mg,0.453486
1,one,0.281195
2,name,0.226609
3,discharge,0.196559
4,significant,0.173714
5,ric,0.172324
6,aggression,0.132329
7,frustration,0.131977
8,discharged,0.127808
9,risk,0.123484


In [ ]:
lowest_attrib_tokens[(lowest_attrib_tokens['tokens'].str.isalpha()) & ~(lowest_attrib_tokens['tokens'].isin(stopwords))][:10].reset_index(drop=True)

,tokens,attribution
0,ayne,-0.477179
1,ĉ,-0.364867
2,plan,-0.246068
3,would,-0.205324
4,cul,-0.177146
5,context,-0.160468
6,medication,-0.138289
7,daily,-0.130862
8,try,-0.130545
9,things,-0.128669


Using this [notebook](https://colab.research.google.com/drive/1SFXcdDASW09re2L_uADwPKbiXJynqPND?usp=sharing), we can get the files to view the aggregated attributions for the entire dataset for both the positive and negative classes. This means we summed up and averaged the attributions for every instance of any given token throughout the entire dataset (whether or not they have positive or negative attributions).

In [ ]:
# df_word = pd.read_csv("/content/drive/MyDrive/cogs402longformer/fakeclinicalnotes/results/notes_attributions/longformer_emb_notes.csv")

Here we see the highest attributions for the positive class, meaning that these tokens have the most influence when the model tries to predict positive. All of these words do have relevence to A.I. related topics.

In [ ]:
# df_word[:15]

Here we see the largest attributions for the negative class, meaning that these tokens have the most influence when the model predicts negative.

In [ ]:
# df_word[:-15:-1]